<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/%D0%A3%D1%80%D0%BE%D0%BA_9_%D0%98%D0%BD%D1%82%D0%B5%D0%B3%D1%80%D0%B0%D1%86%D0%B8%D1%8F_%D0%98%D1%82%D0%BE%D0%B3%D0%BE%D0%B2%D1%8B%D0%B9_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import dill
# from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

In [4]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 1.1 MB/s eta 0:00:00


In [5]:
!python --version

Python 3.10.12


In [6]:
#install python 3.9
!sudo apt-get update -y
!sudo apt-get install python3.9

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

#check python version
!python --version
#3.9.6

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [27.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,357 kB]
Get:13 http://archive.ubuntu.com/ubuntu 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
%cd /content/drive/MyDrive/Colab Notebooks/ML в бизнесе/lesson3/

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/ML в бизнесе/lesson3/'
/content


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

Описание задачи датасета
Определение вероятности наличия сердечно-сосудистых заболеваний по данным первичного осмотра. Нужно предсказать наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра. Датасет сформирован из 70.000 реальных клинических анализов, и в нём используются признаки, которые можно разбить на 3 группы:


Объективные признаки:


Возраст

Рост

Вес

Пол

Результаты измерения:

Артериальное давление верхнее и нижнее

Холестерин

Глюкоза

Субъективные признаки:

Курение

Употребление Алкоголя

Физическая активность

Возраст дан в днях. Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.


Все показатели даны на момент осмотра.


Таргет - наличие сердечно-сосудистых заболеваний (ССЗ)

In [ ]:
data = pd.read_csv('train_case2.csv', ';')
data.head(3)

In [ ]:
df = data.copy()

In [ ]:
df.info()

In [ ]:
df['age_years'] = round(df['age'] / 365).astype(int)

In [ ]:
df.head()

In [ ]:
df.describe().T

In [ ]:
for col in ['height', 'weight', 'ap_hi', 'ap_lo']:

  plt.hist(df[col])
  plt.title(col)
  plt.show()

In [ ]:
for col in ['smoke', 'alco', 'active']:
  sns.countplot(x=col, hue='cardio', data=df)
  plt.title(f'{col} and cardiovascular disease')
  plt.show()

In [ ]:
rcParams['figure.figsize'] = 11, 8
sns.countplot(x='age_years', hue='cardio', data = df, palette="Set2");

In [ ]:
df_categorical = df.loc[:,['cholesterol','gluc', 'smoke', 'alco', 'active']]
sns.countplot(x="variable", hue="value",data= pd.melt(df_categorical));

In [ ]:
df_long = pd.melt(df, id_vars=['cardio'], value_vars=['cholesterol','gluc', 'smoke', 'alco', 'active'])
sns.catplot(x="variable", hue="value", col="cardio",
                data=df_long, kind="count");

In [ ]:
# gender 1 - man, 2 - woman

In [ ]:
df.groupby('gender')['alco'].sum()

In [ ]:
df['cardio'].value_counts(normalize=True)

In [ ]:
pd.crosstab(df['cardio'],df['gender'],normalize=True)

In [ ]:
df['gluc'].value_counts()

In [ ]:
df['cholesterol'].value_counts()

In [ ]:
df.loc[df['weight'] < 40, 'weight'].count()

In [ ]:
df.loc[df['weight'] > 170, 'weight'].count()

In [ ]:
print(df.loc[df['ap_hi'] > 220, 'ap_hi'].count())
print(df.loc[df['ap_hi'] < 80, 'ap_hi'].count())

In [ ]:
df['ap_hi'].quantile([0.05, 0.25, 0.5, 0.75, 0.95])

In [ ]:
print(df.loc[df['ap_lo'] > 110, 'ap_lo'].count())
print(df.loc[df['ap_lo'] < 55, 'ap_lo'].count())

In [ ]:
df['ap_lo'].quantile([0.05, 0.25, 0.5, 0.75, 0.95])

In [ ]:
df.drop(df[(df['height'] > df['height'].quantile(0.975)) | (df['height'] < df['height'].quantile(0.025))].index,inplace=True)
df.drop(df[(df['weight'] > df['weight'].quantile(0.975)) | (df['weight'] < df['weight'].quantile(0.025))].index,inplace=True)

In [ ]:
print("Diastilic pressure is higher than systolic one in {0} cases".format(df[df['ap_lo']> df['ap_hi']].shape[0]))

In [ ]:
df.drop(df[(df['ap_hi'] > 220) | (df['ap_hi'] < 80)].index,inplace=True)
df.drop(df[(df['ap_lo'] > 110) | (df['ap_lo'] < 55)].index,inplace=True)

In [ ]:
df['pulse_pressure'] = round((df['ap_hi'] - df['ap_lo']) / df['ap_hi'], 2)

In [ ]:
df.loc[df['pulse_pressure'] < 0.25, 'pulse_pressure'].value_counts()

In [ ]:
df.drop(df[df['pulse_pressure'] < 0.25].index,inplace=True)

In [ ]:
blood_pressure = df.loc[:,['ap_lo','ap_hi']]
sns.boxplot(x = 'variable',y = 'value',data = blood_pressure.melt())
print("Diastilic pressure is higher than systolic one in {0} cases".format(df[df['ap_lo']> df['ap_hi']].shape[0]))

In [ ]:
df['bmi'] = round(df['weight']/((df['height']/100)**2), 2)
sns.catplot(x="gender", y="bmi", hue="alco", col="cardio", data=df, color = "yellow",kind="box", height=10, aspect=.7);

In [ ]:
corr = df.iloc[:, 1::] .corr()
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,annot = True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

In [ ]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1),
                                                    df['cardio'], random_state=0)

In [ ]:
df.columns

In [ ]:
# Заполнение пропусков
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X


class BMI(BaseEstimator, TransformerMixin):
    def __init__(self, height, weight):
        self.height = height
        self.weight = weight
        self.col = []

    def fit(self, X, y=None):
        self.col = 'bmi'
        return self

    def transform(self, X):
        X[self.col] = X[self.weight].values.reshape(-1,1) / ((X[self.height].values.reshape(-1,1) / 100) ** 2)
        # X_bmi = X[self.weight].values.reshape(-1,1) / ((X[self.height].values.reshape(-1,1) / 100) ** 2)
        # B = pd.DataFrame(data = X_bmi, index = X.index, columns = [self.col])
        return X[self.col].values.reshape(-1,1)


class PulsePressure(BaseEstimator, TransformerMixin):
    def __init__(self, ap_hi, ap_lo):
        self.ap_hi = ap_hi
        self.ap_lo = ap_lo
        self.col = []

    def fit(self, X, y=None):
        self.col = 'pulse_pressure'
        return self

    def transform(self, X):
        X[self.col] = (X[self.ap_hi].values.reshape(-1,1) - X[self.ap_lo].values.reshape(-1,1)) / X[self.ap_hi].values.reshape(-1,1)
        # X_bmi = X[self.weight].values.reshape(-1,1) / ((X[self.height].values.reshape(-1,1) / 100) ** 2)
        # B = pd.DataFrame(data = X_bmi, index = X.index, columns = [self.col])
        return X[self.col].values.reshape(-1,1)


class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]


class OHEEncoderBin(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        B = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        self.columns = B[:1]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        return X[self.columns]

In [ ]:
gluc = Pipeline([
                ('imputer', TextImputer('gluc', '')),
                # ('selector', ColumnSelector(key='gluc')),
                ('selector', NumberSelector(key='gluc')),
                ('standard', StandardScaler())
                # ('ohe', OHEEncoder(key='gluc'))
])
gluc.fit(X_train)
gluc.transform(X_test)

In [ ]:
gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoderBin(key='gender'))
            ])
gender.fit(X_train)
gender.transform(X_test).head(3)

In [ ]:
age =  Pipeline([
                ('imputer', TextImputer('age_years', '')),
                ('selector', NumberSelector(key='age_years')),
                ('standard', StandardScaler())
            ])
age.fit(X_train)
age.transform(X_test)

In [ ]:
bmi =  Pipeline([
                ('bmi', BMI('height', 'weight')),
                ('standard', StandardScaler())
            ])
bmi.fit(df)
bmi.transform(X_test)

In [ ]:
pulse_pressure =  Pipeline([
                ('pulse', BMI('ap_hi', 'ap_lo')),
                ('standard', StandardScaler())
            ])
pulse_pressure.fit(df)
pulse_pressure.transform(X_test)

In [ ]:
continuos_cols = ['age_years', 'ap_hi', 'ap_lo', 'height'] # ['age_years', 'ap_hi', 'ap_lo', 'height', 'weight'] #
cat_cols = ['gluc', 'cholesterol']
base_cols = ['alco', 'active', 'smoke', 'gender']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('imputer', TextImputer(cont_col, 0)),
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))

for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('imputer', TextImputer(cat_col, 0)),
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    cat_transformers.append((cat_col, cat_transformer))

for base_col in base_cols:
    base_transformer = Pipeline([
                ('imputer', TextImputer(base_col, 0)),
                ('selector', NumberSelector(key=base_col)),
                ('one', OHEEncoderBin(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

bmi =  Pipeline([
                ('bmi', BMI('height', 'weight')),
                ('standard', StandardScaler())
            ])
continuos_transformers.append(('bmi', bmi))

pulse_pressure =  Pipeline([
                ('pulse', BMI('ap_hi', 'ap_lo')),
                ('standard', StandardScaler())
            ])
continuos_transformers.append(('pulse_pressure', pulse_pressure))

In [ ]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train).shape

In [ ]:
import random
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from catboost import CatBoostClassifier

In [ ]:
random.seed(0)
models =  [xgb.XGBClassifier(), LogisticRegression(), CatBoostClassifier(verbose=0)]

In [ ]:
preds = []
for model in models:
  pipeline = Pipeline([
    ('features',feats),
    ('classifier', model)
    ])
  pipeline.fit(X_train, y_train)
  preds.append(pipeline.predict_proba(X_test)[:, 1])

In [ ]:
#наши прогнозы для тестовой выборки
preds

In [ ]:
result = {
    'model': [],
    'Best Threshold': [],
    'F-Score': [],
    'Precision': [],
    'Recall': []
}

In [ ]:
fscore = []
ix = []
thresholds = []
for i in range(len(preds)):
  precision, recall, threshold = precision_recall_curve(y_test, preds[i])
  thresholds.append(threshold)
  fscore.append((2 * precision * recall) / (precision + recall))
  # locate the index of the largest f score
  ix.append(np.argmax(fscore[i]))
  print('Best params for ', models[i], ': Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[i][ix[i]],
                                                                          fscore[i][ix[i]],
                                                                          precision[ix[i]],
                                                                          recall[ix[i]]))
  result['model'].append(models[i])
  result['Best Threshold'].append(thresholds[i][ix[i]])
  result['F-Score'].append(fscore[i][ix[i]])
  result['Precision'].append(precision[ix[i]])
  result['Recall'].append(recall[ix[i]])

In [ ]:
result = pd.DataFrame(data=result).sort_values('F-Score', ascending=False)
result

In [ ]:
def plot_roc_curve(y_test, y_score):
  sns.set(font_scale=1.5)
  sns.set_color_codes("muted")

  plt.figure(figsize=(10, 8))
  fpr, tpr, thresholds_ = roc_curve(y_test, y_score, pos_label=1)
  lw = 2
  plt.plot(fpr, tpr, lw=lw, label='ROC curve ')
  plt.plot([0, 1], [0, 1])
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('ROC curve')
  plt.savefig("ROC.png")
  plt.show()

In [ ]:
plot_roc_curve(y_test, preds[0])

In [ ]:
from sklearn.metrics import roc_auc_score, log_loss
for i, m in enumerate(preds):
  roc_auc_scor = roc_auc_score(y_true=y_test, y_score=preds[i])
  print(models[i], ':')
  print("roc auc score: {}".format(roc_auc_score(y_true=y_test, y_score=preds[i])))
  print("log loss score: {}".format(log_loss(y_true=y_test, y_pred=preds[i])))
  print('_________________________________________________________________')

In [ ]:
#мы уже нашли ранее "оптимальный" порог, когда максимизировали f_score
font = {'size' : 15}

plt.rc('font', **font)

for i in range(len(preds)):
  cnf_matrix = confusion_matrix(y_test, preds[i]>thresholds[i][ix[i]])
  plt.figure(figsize=(10, 8))
  plot_confusion_matrix(cnf_matrix, classes=['cardio_0', 'cardio_1'],
                      title='Confusion matrix ' + str(models[i]))
# plt.savefig("conf_matrix.png")
  plt.show()

In [ ]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', CatBoostClassifier(verbose=0))
    ])
pipeline.fit(X_train, y_train)
preds = pipeline.predict_proba(X_test)[:, 1]

In [ ]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, preds)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix],
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

In [ ]:
plot_roc_curve(y_test, preds)

In [ ]:
font = {'size' : 15}

plt.rc('font', **font)

cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
plt.figure(figsize=(10, 8))
plot_confusion_matrix(cnf_matrix, classes=['cardio_0', 'cardio_1'],
                      title='Confusion matrix')
plt.savefig("conf_matrix.png")
plt.show()

In [ ]:
# pipeline.get_params()

In [ ]:
with open("Catboost_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

In [ ]:
import dill as pickle
with open("Catboost_pipeline.pkl", 'wb') as file:
    pickle.dump(pipeline, file)

In [ ]:
model = pickle.load(open('Catboost_pipeline.pkl', 'rb'))

In [ ]:
x = pd.DataFrame({"age_years": [55, 66, 33],
                                  'gender': [2, 2, 1],
                                  'height': [180, 140, 170],
                                  'weight': [110, 50, 80],
                                  'ap_hi': [160, 120, 120],
                                  'ap_lo': [120, 80, 80],
                                  'cholesterol': [2, 2, 1],
                                  'gluc': [1, 2, 3],
                                  'smoke': [1, 1, 1],
                                  'alco': [1, 1, 1],
                                  'active': [1, 1, 1]
                        })
x

In [ ]:
model.predict_proba(x)